# NoteBook goal to get the best variation of Domain Model and ML Task model after tuning
<b>it will be used only once</b>

In [48]:
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import importlib
import Embeddings
importlib.reload(Embeddings)
from Embeddings import Embedders_Five, ModelDownloader
import load_datasets
importlib.reload(load_datasets)
from load_datasets import LoadDatasets
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import make_scorer, accuracy_score
import pandas as pd
import numpy as np
import pickle
from sklearn.pipeline import Pipeline
import os
import random

In [8]:

# Load The targets of all dataset problems
loader = LoadDatasets()
domain_data = loader.load_domain()
ml_task_data = loader.load_mltask()  

# User stories Loading
user_stories = pd.read_excel("./Dataset/Domain_Classification_Data/Synthetic User Stories.xlsx")
user_stories['Domain'] = user_stories['Domain'].str.lower()
embedder = Embedders_Five(user_stories["User Story"])

# Get embeddings
x_embeddings = {
    "GloVE": np.array(embedder.getGloVEEmbedding()),
    "BERT": np.array(embedder.getBERTEmbeddings())
}

y_target = {
    "Domain": np.array(domain_data),
    "ML_Task": np.array(ml_task_data)
}


# Getting Best BERT - XGBoost for Domain

In [12]:
xgb_model = xgb.XGBClassifier(
    tree_method='hist',
    device='cuda',
    max_bins=256,
    early_stopping_rounds=2
)

X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
    x_embeddings["BERT"], 
    y_target["Domain"], 
    test_size=0.2, 
    random_state=42
)

params = {
    'min_child_weight': np.arange(1, 7, 2),  # default is 1
    'gamma': np.arange(0.01, 0.2, 0.05),     # default is 0
    'subsample': np.arange(0.6, 1, 0.1),     # default is 1
    'max_depth': np.arange(2, 8),            # default is 6
    'learning_rate': np.arange(0.01, 0.41, 0.05),  # default is 0.3
    'n_estimators': [100, 200, 300],
    'max_leaves': [0, 32, 64],
    'max_bin': [256],
    'tree_method': ['hist'],                 
    'sampling_method': ['gradient_based'],
}

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=params,
    n_iter=20,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=5,
    pre_dispatch='2*n_jobs',
    scoring='accuracy'
)

random_search.fit(
    X_train_bert, 
    y_train_bert,
    eval_set=[(X_test_bert, y_test_bert)],
    verbose=True
)

print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

os.makedirs('./MLModels', exist_ok=True)
best_model = random_search.best_estimator_
model_path = './MLModels/xgboost_best_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model, f)

print(f"Model saved to {model_path}")


best_params = random_search.best_params_.copy()
best_params.pop('tree_method', None)  # Remove tree_method to avoid conflict

final_model = xgb.XGBClassifier(
    **best_params,
    tree_method='hist',
    device='cuda',  
    early_stopping_rounds=2,
)

final_model.fit(
    X_train_bert,
    y_train_bert,
    eval_set=[(X_test_bert, y_test_bert)],
    verbose=True
)

final_model_path = './MLModels/xgboost_final_model.pkl'
with open(final_model_path, 'wb') as f:
    pickle.dump(final_model, f)

print(f"Final model saved to {final_model_path}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:16:36] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are not used.

  warnings.warn(smsg, UserWarning)
/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:16:36] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are not used.

  warnings.warn(smsg, UserWarning)
/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:16:36] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are 

[0]	validation_0-mlogloss:3.48381
[0]	validation_0-mlogloss:3.48430
[0]	validation_0-mlogloss:3.48381
[0]	validation_0-mlogloss:3.48428
[0]	validation_0-mlogloss:3.48423
[1]	validation_0-mlogloss:3.35131
[1]	validation_0-mlogloss:3.35209
[1]	validation_0-mlogloss:3.35258
[1]	validation_0-mlogloss:3.35281
[1]	validation_0-mlogloss:3.35115
[2]	validation_0-mlogloss:3.23412
[2]	validation_0-mlogloss:3.23538
[2]	validation_0-mlogloss:3.23668
[2]	validation_0-mlogloss:3.23587
[2]	validation_0-mlogloss:3.23466
[3]	validation_0-mlogloss:3.12989
[3]	validation_0-mlogloss:3.13220
[3]	validation_0-mlogloss:3.13144
[3]	validation_0-mlogloss:3.13249
[3]	validation_0-mlogloss:3.13059
[4]	validation_0-mlogloss:3.03603
[4]	validation_0-mlogloss:3.03863
[4]	validation_0-mlogloss:3.03827
[4]	validation_0-mlogloss:3.03890
[4]	validation_0-mlogloss:3.03690
[5]	validation_0-mlogloss:2.95075
[5]	validation_0-mlogloss:2.95318
[5]	validation_0-mlogloss:2.95300
[5]	validation_0-mlogloss:2.95374
[5]	validation

/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:17:39] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[99]	validation_0-mlogloss:0.90243
[99]	validation_0-mlogloss:0.90543


...

[12]	validation_0-mlogloss:0.11253
[96]	validation_0-mlogloss:0.10898
[96]	validation_0-mlogloss:0.11889
[17]	validation_0-mlogloss:0.11067
[CV] END gamma=0.16000000000000003, learning_rate=0.16000000000000003, max_bin=256, max_depth=2, max_leaves=64, min_child_weight=1, n_estimators=100, sampling_method=gradient_based, subsample=0.6, tree_method=hist; total time=  51.8s
[0]	validation_0-mlogloss:0.35104
[97]	validation_0-mlogloss:0.10870
[13]	validation_0-mlogloss:0.11075
[97]	validation_0-mlogloss:0.11895
[18]	validation_0-mlogloss:0.11035
[98]	validation_0-mlogloss:0.10835
[98]	validation_0-mlogloss:0.11873
[1]	validation_0-mlogloss:0.27838
[14]	validation_0-mlogloss:0.10919
[19]	validation_0-mlogloss:0.10940
[99]	validation_0-mlogloss:0.10802
[99]	validation_0-mlogloss:0.11860
[2]	validation_0-mlogloss:0.23828
[15]	validation_0-mlogloss:0.10807
[20]	validation_0-mlogloss:0.10918


/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:36:00] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are not used.

  warnings.warn(smsg, UserWarning)
/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:36:00] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are not used.

  warnings.warn(smsg, UserWarning)


[3]	validation_0-mlogloss:0.20802
[16]	validation_0-mlogloss:0.10748
[21]	validation_0-mlogloss:0.10892
[CV] END gamma=0.16000000000000003, learning_rate=0.16000000000000003, max_bin=256, max_depth=2, max_leaves=64, min_child_weight=1, n_estimators=100, sampling_method=gradient_based, subsample=0.6, tree_method=hist; total time=  52.0s
[0]	validation_0-mlogloss:0.34293
[CV] END gamma=0.16000000000000003, learning_rate=0.16000000000000003, max_bin=256, max_depth=2, max_leaves=64, min_child_weight=1, n_estimators=100, sampling_method=gradient_based, subsample=0.6, tree_method=hist; total time=  51.9s
[0]	validation_0-mlogloss:0.34604
[17]	validation_0-mlogloss:0.10720
[22]	validation_0-mlogloss:0.10838
[4]	validation_0-mlogloss:0.18620
[1]	validation_0-mlogloss:0.27615
[1]	validation_0-mlogloss:0.27367
[23]	validation_0-mlogloss:0.10813
[18]	validation_0-mlogloss:0.10741
[5]	validation_0-mlogloss:0.16897
[2]	validation_0-mlogloss:0.23210
[2]	validation_0-mlogloss:0.23231
[24]	validation_

/localscratch/ahmedyra.50815683.0/env/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:36:13] WARNING: /shared_tmp/build_wheels_tmp.2163411/python-3.10/xgboost/src/learner.cc:740: 
Parameters: { "max_bins" } are not used.

  warnings.warn(smsg, UserWarning)


[3]	validation_0-mlogloss:0.44418
[4]	validation_0-mlogloss:0.37873
[5]	validation_0-mlogloss:0.32660
[6]	validation_0-mlogloss:0.28399
[7]	validation_0-mlogloss:0.24964
[8]	validation_0-mlogloss:0.22215
[9]	validation_0-mlogloss:0.19929
[10]	validation_0-mlogloss:0.18033
[11]	validation_0-mlogloss:0.16486
[12]	validation_0-mlogloss:0.15255
[13]	validation_0-mlogloss:0.14248
[14]	validation_0-mlogloss:0.13441
[15]	validation_0-mlogloss:0.12823
[16]	validation_0-mlogloss:0.12269
[17]	validation_0-mlogloss:0.11857
[18]	validation_0-mlogloss:0.11512
[19]	validation_0-mlogloss:0.11223
[20]	validation_0-mlogloss:0.11032
[21]	validation_0-mlogloss:0.10836
[22]	validation_0-mlogloss:0.10679
[23]	validation_0-mlogloss:0.10546
[24]	validation_0-mlogloss:0.10436
[25]	validation_0-mlogloss:0.10385
[26]	validation_0-mlogloss:0.10338
[27]	validation_0-mlogloss:0.10274
[28]	validation_0-mlogloss:0.10259
[29]	validation_0-mlogloss:0.10218
[30]	validation_0-mlogloss:0.10190
[31]	validation_0-mlogloss:

# Training GLOVE LINEAR - SVC - LP for ML Task
<ul>
    <li>Till now there is no big support for multi-label classificatio</li>
    <li>We tested Grid Search, and Randomized Search, it was not possible</li>
</ul>

In [28]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(
   x_embeddings["GloVE"], 
   y_target["ML_Task"], 
   test_size=0.2, 
   random_state=42
)

classifier = LinearSVC(C=0.1,loss = 'hinge', penalty = 'l2',random_state=42)
model = LabelPowerset(classifier=classifier)
model.fit(X_train_glove, y_train_glove)

os.makedirs('./MLModels', exist_ok=True)
model_path = './MLModels/linear_svc_final_model.pkl'
with open(model_path, 'wb') as f:
   pickle.dump(model, f)

print(f"Model saved to {model_path}")

Model saved to ./MLModels/linear_svc_final_model.pkl


# Testing both Models

In [55]:
from sklearn.metrics import accuracy_score

# Load models
with open('./MLModels/xgboost_best_model.pkl', 'rb') as f:
   xgb_model = pickle.load(f)
with open('./MLModels/linear_svc_final_model.pkl', 'rb') as f:
   svc_model = pickle.load(f)

# Get 10 random samples
idx = random.sample(range(len(user_stories)), 10)
samples = user_stories.iloc[idx]
true_domains = domain_data.iloc[idx]
true_tasks = ml_task_data.iloc[idx]

# Get embeddings
test_stories = samples['User Story'].values
embedder = Embedders_Five(test_stories)
bert_samples = embedder.getBERTEmbeddings()
glove_samples = embedder.getGloVEEmbedding()

# Get predictions
bert_preds = [xgb_model.predict(sample.reshape(1, -1))[0] for sample in bert_samples]
glove_preds = [svc_model.predict(sample.reshape(1, -1)).toarray()[0] for sample in glove_samples]

# Calculate accuracy
bert_accuracy = accuracy_score(true_domains, bert_preds)
glove_accuracy = accuracy_score(np.array(true_tasks), glove_preds)

print(f"BERT Domain Classification Accuracy: {bert_accuracy:.2%}")
print(f"GloVE Task Classification Accuracy: {glove_accuracy:.2%}")

# Print some examples
print("\nSample Predictions:")
for i in range(3):  # Show first 3 examples
   print(f"\nStory: {test_stories[i]}")
   print(f"True Domain: {true_domains.iloc[i]}, Predicted Domain: {bert_preds[i]}")
   print(f"True Tasks: {np.array(true_tasks)[i][0]}, Predicted Tasks: {glove_preds[i]}")

BERT Domain Classification Accuracy: 100.00%
GloVE Task Classification Accuracy: 70.00%

Sample Predictions:

Story: As a psychologist, I want a machine augmented intelligence tool that can analyze patient data and provide me with more accurate predictions on potential mental health issues and treatment options.
True Domain: Target    19
Name: 5812, dtype: int64, Predicted Domain: 19
True Tasks: 0, Predicted Tasks: [0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0]

Story: As a musician, I want to use spoken dialog systems to create more interactive and personalized music performances, so that I can engage audiences and create unique musical experiences.
True Domain: Target    34
Name: 11501, dtype: int64, Predicted Domain: 34
True Tasks: 0, Predicted Tasks: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0]

Story: As a computer vision researcher, I want to use cost-sensitive learning to develop models that can accurately predict and address potential biases in object recognitio